In [18]:
from datasets import load_dataset
from transformers import Trainer, TrainingArguments
from transformers import RobertaTokenizer, RobertaForSequenceClassification

In [19]:
ds = load_dataset("Jiann/STORAL")

In [20]:
moral = []
moral += ds["storal_en_train"]
moral += ds["storal_en_test"]
moral += ds["storal_en_valid"]

In [25]:
train_x = []
train_y = []

import random

for x in range(len(ds["storal_en_train"])):
    train_x.append(ds["storal_en_train"][x]['story'] + "the moral of this story is:" + ds["storal_en_train"][x]['moral'])
    train_y.append(1)
  
for x in range(len(ds["storal_en_train"])):

    i = random.sample(range(1, len(ds["storal_en_train"])), 2)
    
    train_x.append(ds["storal_en_train"][i[0]]['story'] + "the moral of this story is:" + ds["storal_en_train"][i[1]]['moral'])
    train_y.append(0)
  

In [26]:
test_x = []
test_y = []

for x in range(len(ds["storal_en_test"])):
    test_x.append(ds["storal_en_test"][x]['story'] + "the moral of this story is:" + ds["storal_en_test"][x]['moral'])
    test_y.append(1)
  
for x in range(len(ds["storal_en_test"])):

    i = random.sample(range(1, len(ds["storal_en_test"])), 2)
    
    test_x.append(ds["storal_en_test"][i[0]]['story'] + "the moral of this story is:" + ds["storal_en_test"][i[1]]['moral'])
    test_y.append(0)
  

In [27]:
from datasets import Dataset
data_dict_list = [
    {"text": t, "label": l} for t, l in zip(train_x, train_y)
]

# 3. Create the Hugging Face Dataset from the list of dictionaries
dataset = Dataset.from_list(data_dict_list)

data_dict_list = [
    {"text": t, "label": l} for t, l in zip(test_x, test_y)
]

# 3. Create the Hugging Face Dataset from the list of dictionaries
test_dataset = Dataset.from_list(data_dict_list)


In [29]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base')
# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)
tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)
# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Trainer API for fine-tuning
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_test_dataset
)
# Fine-tune the model
trainer.train()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|███████████████████████████| 712/712 [00:00<00:00, 1142.95 examples/s]
/home/kuroro/py_envs/lib/python3.12/site-packages/torch/cuda/__init__.py:827: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/kuroro/py_envs/lib/python3.12/site-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss


TrainOutput(global_step=402, training_loss=0.6946937219420476, metrics={'train_runtime': 13500.9416, 'train_samples_per_second': 0.475, 'train_steps_per_second': 0.03, 'total_flos': 1686015642746880.0, 'train_loss': 0.6946937219420476, 'epoch': 3.0})

In [41]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = trainer.model
# Create a pipeline for text classification
classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)
# Test the classifier on some text
result = classifier("RoBERTa is an amazing language model!")
print(result)

Device set to use cpu


[{'label': 'LABEL_1', 'score': 0.5034279823303223}]


In [53]:
test_x = []
test_y = []

for x in range(len(ds["storal_en_test"])):
    test_x.append(ds["storal_en_test"][x]['story'] + "the moral of this story is:" + ds["storal_en_test"][x]['moral'])
    test_y.append(1)
  
for x in range(len(ds["storal_en_test"])):

    i = random.sample(range(1, len(ds["storal_en_test"])), 2)
    
    test_x.append(ds["storal_en_test"][i[0]]['story'] + "the moral of this story is:" + ds["storal_en_test"][i[1]]['moral'])
    test_y.append(0)
  

In [69]:
result_list = 0

for i in range(len(test_x)):
    if len(test_x[i]) > 511:
        test_x[i] = test_x[i][:511]
    r = classifier(test_x[i])[0]['score']
    if r > 0.5 and test_y[i] == 1:
        result_list += 1
    elif r < 0.5 and test_y[i] == 0:
        result += 1

result_list/len(test_x)

0.5